In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from qtcodes import RotatedDecoder
from qtcodes import RepetitionQubit

In [2]:
from qiskit_aer.noise import NoiseModel
from qiskit_aer.noise.errors import pauli_error, depolarizing_error
from qiskit_aer import Aer

def get_noise_model(p_err):

    error_gate1 = pauli_error([("X", p_err / 2), ("Z", p_err / 2), ("I", 1 - p_err)])

    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error_gate1, "id")
    return noise_model

In [3]:
p_err = 0.10
shots = 10000


def get_results(circ):
    results = Aer.get_backend("aer_simulator").run(circ, 
                      noise_model=get_noise_model(p_err),
                      optimization_level=0,
                      shots=shots,
                     ).result().get_counts()
    return results

In [4]:
d = 3
T = 1

decoder = RotatedDecoder({"d":d,"T":T})

In [5]:
qubit = RepetitionQubit({'d':d})
qubit.reset_z()
qubit.stabilize()
qubit.id_data()
qubit.stabilize()
qubit.readout_z()
decoder.set_look_up_table("Z",0,get_results(qubit.circ))

qubit = RepetitionQubit({'d':d})
qubit.reset_z()
qubit.x()
qubit.stabilize()
qubit.id_data()
qubit.stabilize()
qubit.readout_z()
decoder.set_look_up_table("Z",1,get_results(qubit.circ))

In [6]:
decoder.look_up_table

{'X': {0: {}, 1: {}},
 'Z': {0: {'0 10 00': 466,
   '0 00 00': 8546,
   '1 01 00': 472,
   '0 11 00': 446,
   '0 01 00': 23,
   '1 10 00': 29,
   '1 11 00': 18},
  1: {'1 10 00': 422,
   '1 00 00': 8612,
   '0 11 00': 22,
   '0 01 00': 445,
   '1 11 00': 456,
   '1 01 00': 18,
   '0 00 00': 2,
   '0 10 00': 23}}}

In [7]:
readout_string = "1 01 00"
readout_type = "Z"
print(decoder._run_look_up(readout_string, readout_type))
decoder.correct_readout_look_up_table(readout_string, readout_type)

[0.9632653061224489, 0.03673469387755102]


0